```
使用bank-additional-full数据集，进行数据分析和建模预测
包含Hyperparameter tuning (HP) Feature engineering (FE)
利用3个模型进行预测
    Logistic Classifier
    XGBoost
    SVC
执行程序，预测结果显示
bank-additional-fullデータセットを使用してデータ分析とモデリング予測を行う
ハイパーパラメータチューニング（HP）特徴エンジニアリング（FE）が含まれています
3つのモデルを使用して予測を行う
     Logistic Classifier
     XGBoost
     SVC
プログラムを実行し、予測結果を表示する
```

### import package and setting parameter

In [ ]:
import os
import numpy as np 
import pandas as pd
import missingno as msno
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import phik

# Modeling libraries
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

# Other settings
sns.set_style("whitegrid")
sns.set(rc={'figure.figsize':(10,8)})
# plt.rcParams['font.sans-serif'] = ['SimHei']
# plt.rcParams['axes.unicode_minus'] = False 
# sns.set(font='SimHei')
matplotlib.rc('font', family='TakaoPGothic')
sns.set_context("paper", font_scale=1.1)
# ignore warning for axis logger of matplotlib
matplotlib.axes._axes._log.setLevel('ERROR')
%matplotlib inline

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# set basically parameter
path='/kaggle/input/bank-marketing/bank-additional-full.csv'

# Set the number of rows and columns displayed
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100


In [ ]:
base_info=pd.read_csv(path,sep = ';')
# 予測結果の列はyに保存されます（预测结果列保存到y）
y = pd.get_dummies(base_info['y'], columns = ['y'], prefix = ['y'], drop_first = True)
base_info.head()

# データ分析 Data Analysis(EDA)

In [ ]:
base_info.columns.values,len(base_info.columns.values)

In [ ]:
# 欠損値がないかどうかを確認します（确认是否有missing丢失值）
msno.matrix(base_info,labels=True)

In [ ]:
base_info.info()

### 現在、与えられたデータセットのほとんどはカテゴリーフィールドであり、モデルトレーニングを直接実行することはできず、数値タイプに変換する必要があります
当前给定数据集大部分都是分类字段，不能直接进行模型训练，需要转换成数值类型
```
根据提示，数据分成多类分别进行数据理解和数据分析（According to the prompts, the data is divided into multiple categories for data understanding and data analysis）
客户信息部分（client）
   1 - age (numeric)
   2 - job : type of job (categorical: "admin.","blue-collar","entrepreneur","housemaid","management","retired","self-employed","services","student","technician","unemployed","unknown")
   3 - marital : marital status (categorical: "divorced","married","single","unknown"; note: "divorced" means divorced or widowed)
   4 - education (categorical: "basic.4y","basic.6y","basic.9y","high.school","illiterate","professional.course","university.degree","unknown")
   5 - default: has credit in default? (categorical: "no","yes","unknown")
   6 - housing: has housing loan? (categorical: "no","yes","unknown")
   7 - loan: has personal loan? (categorical: "no","yes","unknown")
当前广告活动最后联系相关信息（related with the last contact of the current campaign）
   8 - contact: contact communication type (categorical: "cellular","telephone") 
   9 - month: last contact month of year (categorical: "jan", "feb", "mar", ..., "nov", "dec")
  10 - day_of_week: last contact day of the week (categorical: "mon","tue","wed","thu","fri")
  11 - duration: last contact duration, in seconds (numeric). 
其他属性（other attributes）
  12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
  13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
  14 - previous: number of contacts performed before this campaign and for this client (numeric)
  15 - poutcome: outcome of the previous marketing campaign (categorical: "failure","nonexistent","success")
社会和经济状况属性（social and economic context attributes）
  16 - emp.var.rate: employment variation rate（就业变化率） - quarterly indicator (numeric)
  17 - cons.price.idx: consumer price index（消费者价格指数） - monthly indicator (numeric)     
  18 - cons.conf.idx: consumer confidence index（消费者信心指数） - monthly indicator (numeric)     
  19 - euribor3m: euribor 3 month rate（欧元3个月利率） - daily indicator (numeric)
  20 - nr.employed: number of employees（在职员工人数） - quarterly indicator (numeric)
```

# クライアント情報のデータ理解（client）

In [ ]:
client_info=base_info.iloc[:,0:7]
client_info.head()

In [ ]:
print("Job \n {}\n {}".format(len(client_info['job'].unique()),client_info['job'].unique()))

In [ ]:
print("Marital \n {}\n {}".format(len(client_info['marital'].unique()),client_info['marital'].unique()))

In [ ]:
print("Education \n {}\n {}".format(len(client_info['education'].unique()),client_info['education'].unique()))

In [ ]:
print("Default \n {}\n {}".format(len(client_info['default'].unique()),client_info['default'].unique()))

In [ ]:
print("Housing \n {}\n {}".format(len(client_info['housing'].unique()),client_info['housing'].unique()))

In [ ]:
print("Loan \n {}\n {}".format(len(client_info['loan'].unique()),client_info['loan'].unique()))

### 年齢フィールドのデータ理解

In [ ]:
base_info['age'].describe()

In [ ]:
sns.boxplot(x=base_info["age"],data=base_info)

In [ ]:
fig,ax=plt.subplots()
fig.set_size_inches(15,8)
sns.countplot(x='age',data=base_info)
ax.set_xlabel("Age",fontsize=15)
ax.set_ylabel("Count",fontsize=15)
ax.set_title("Age count distribution chart",fontsize=15)

In [ ]:
# 计算outlier范围higher_outlier和lower_outlier（Calculate the outlier range higher_outlier and lower_outlier）
print(client_info['age'].describe())
higher_outlier=client_info['age'].quantile(q = 0.75)+1.5*(client_info['age'].quantile(q = 0.75)-client_info['age'].quantile(q = 0.25))
lower_outlier=client_info['age'].quantile(q = 0.25)-1.5*(client_info['age'].quantile(q = 0.75)-client_info['age'].quantile(q = 0.25))
print("higher_outlier={}\nlower_outlier={}".format(higher_outlier,lower_outlier))

外れ値比率

In [ ]:
outlier_count=client_info[client_info['age']>higher_outlier]['age'].count()
print("outlier count={}".format(outlier_count))
print("outlier rate={} %".format(round(outlier_count*100/len(client_info),2)))

### JOBフィールドのデータ理解
職種毎に人数統計

In [ ]:
fig,ax=plt.subplots()
fig.set_size_inches(15,6)
sns.countplot(client_info['job'])
ax.set_xlabel("Job",fontsize=10)
ax.set_ylabel("Count",fontsize=10)
ax.set_title("Job distribution chart")
sns.despine()

### Marial,Eductionなどのフィールドステータスの確認

In [ ]:
fig,ax=plt.subplots()
fig.set_size_inches(10,6)
sns.countplot(client_info['marital'])
ax.set_xlabel("marital",fontsize=10)
ax.set_ylabel("Count",fontsize=10)
ax.set_title("marital distribution chart")
sns.despine()

In [ ]:
fig,ax=plt.subplots()
fig.set_size_inches(15,6)
sns.countplot(client_info['education'])
ax.set_xlabel("education",fontsize=10)
ax.set_ylabel("Count",fontsize=10)
ax.set_title("education distribution chart")
sns.despine()

In [ ]:
fig,(ax1,ax2,ax3)=plt.subplots(nrows=1,ncols=3,figsize=(20,6))
sns.countplot(x='default',ax=ax1,data=client_info,order = ['yes', 'no', 'unknown'])
ax1.set_title("default columns data distribution chart",fontsize=15)
sns.countplot(x='housing',ax=ax2,data=client_info,order = ['yes', 'no', 'unknown'])
ax2.set_title("housing columns data distribution chart",fontsize=15)
sns.countplot(x='loan',ax=ax3,data=client_info,order = ['yes', 'no', 'unknown'])
ax3.set_title("loan columns data distribution chart",fontsize=15)

In [ ]:
# ['default']カラムについて['yes']、['no']、['unknown']になるそれぞれ種類の人数統計
print("number of 'yes' in default={}\n".format(client_info[client_info['default']=='yes']['age'].count()))
print("number of 'no' in default={}\n".format(client_info[client_info['default']=='no']['age'].count()))
print("number of 'unknown' in default={}\n".format(client_info[client_info['default']=='unknown']['age'].count()))

In [ ]:
# ['housing']カラムについて['yes']、['no']、['unknown']になるそれぞれ種類の人数統計
print("number of 'yes' in housing={}\n".format(client_info[client_info['housing']=='yes']['age'].count()))
print("number of 'no' in housing={}\n".format(client_info[client_info['housing']=='no']['age'].count()))
print("number of 'unknown' in housing={}\n".format(client_info[client_info['housing']=='unknown']['age'].count()))

In [ ]:
# ['loan']カラムについて['yes']、['no']、['unknown']になるそれぞれ種類の人数統計
print("number of 'yes' in loan={}\n".format(client_info[client_info['loan']=='yes']['age'].count()))
print("number of 'no' in loan={}\n".format(client_info[client_info['loan']=='no']['age'].count()))
print("number of 'unknown' in loan={}\n".format(client_info[client_info['loan']=='unknown']['age'].count()))

In [ ]:
# 从下图结果发现，不订阅的人数远远大于订阅的人数
# サービス利用している人と利用していない人と比較する、利用していない人が多いのを見える。
# From the result of the figure below, it is found that the number of people who do not subscribe is far greater than the number of people who subscribe
fig,ax=plt.subplots()
fig.set_size_inches(8,5)
sns.countplot(base_info['y'])
# ax.set_xlabel("education",fontsize=10)
# ax.set_ylabel("Count",fontsize=10)
ax.set_title("y column distribution chart")
sns.despine()

In [ ]:
# 订阅的人的年龄分布状况（Age distribution of subscribers）
# 利用者の年齢層表示、チャートから25歳から40歳までの利用者が多いのを見える。
fig,ax=plt.subplots()
fig.set_size_inches(15,8)
sns.countplot(x='age',data=base_info[base_info['y']=='yes'])
ax.set_title("subscriptioner's age distribution chart",fontsize=15)

In [ ]:
# 根据下图发现订阅主要集中在30到50岁的人群，不订阅集中在32到47岁的人群
# チャートから利用している人は30代ー50代まで、利用していない人は32歳ー47歳まで集中しているのを見える
# （According to the figure below, subscriptions are mainly concentrated in people aged 30 to 50, and non-subscriptions are concentrated in people aged 32 to 47.）
plt.rcParams['figure.figsize'] = 10,5
sns.boxplot(x='y', y='age', data=base_info)
plt.title("Correlation between y column and age",fontsize=15)
plt.show()

In [ ]:
# 不同年龄段和y列的相关性表示（Correlation representation of different age groups and y column）
from matplotlib.colors import LogNorm
sns.heatmap(base_info.groupby(["age", "y"]).size().unstack(),norm=LogNorm())

In [ ]:
# 根据heatmap展示y列和那些字段有相关性（According to the heatmap, the y column is related to those fields）
# ｙ列で色が深ければ深いほど相関性が高くなる。
fig, ax = plt.subplots(figsize=(10, 10))
sns.heatmap(base_info.phik_matrix(), vmin=-1, vmax=1, cmap='Blues', cbar=True, annot=False, ax=ax)

In [ ]:
# 文字列から数値に変更する
client_info_encoder=client_info.copy()
# 标签编码转换，将现在的字符转换成数字(Label encoding conversion, convert current characters into numbers)
labelencoder = LabelEncoder()
client_info_encoder['job']      = labelencoder.fit_transform(client_info_encoder['job']) 
client_info_encoder['marital']  = labelencoder.fit_transform(client_info_encoder['marital']) 
client_info_encoder['education']= labelencoder.fit_transform(client_info_encoder['education']) 
client_info_encoder['default']  = labelencoder.fit_transform(client_info_encoder['default']) 
client_info_encoder['housing']  = labelencoder.fit_transform(client_info_encoder['housing']) 
client_info_encoder['loan']     = labelencoder.fit_transform(client_info_encoder['loan']) 

In [ ]:
client_info_encoder.head()

In [ ]:
# 年齢グループに分けて
#function to creat group of ages, this helps because we have 78 differente values here
def age(dataframe):
    dataframe.loc[dataframe['age'] <= 32, 'age'] = 1
    dataframe.loc[(dataframe['age'] > 32) & (dataframe['age'] <= 47), 'age'] = 2
    dataframe.loc[(dataframe['age'] > 47) & (dataframe['age'] <= 70), 'age'] = 3
    dataframe.loc[(dataframe['age'] > 70) & (dataframe['age'] <= 98), 'age'] = 4
           
    return dataframe

age(client_info_encoder);

```
年齢グループに分けて
age group1:age<32
age group2:age>32 and age<=47
age group3:age>47 and age<=70
age group4:age>70 and age<=98
```

In [ ]:
# print(client_info_encoder['age'].unique())
fig,ax=plt.subplots(figsize=(10,6))
sns.countplot(x='age',data=client_info_encoder,order = [1, 2, 3, 4])
ax.set_title("Age group distribution status")
ax.set_xlabel("Age group")

# 現在のキャンペーンの最後の連絡先に関連する（related with the last contact of the current campaign）

In [ ]:
related_info=base_info.iloc[:,7:11]

In [ ]:
related_info.head()

In [ ]:
# 電話で通話時間長さ統計
fig,ax=plt.subplots()
fig.set_size_inches(10,6)
sns.countplot(x='duration',data=related_info)

In [ ]:
# 外れ値範囲計算
# 计算outlier范围higher_outlier和lower_outlier（Calculate the outlier range higher_outlier and lower_outlier）
print(related_info['duration'].describe())
higher_outlier=related_info['duration'].quantile(q = 0.75)+1.5*(related_info['duration'].quantile(q = 0.75)-related_info['duration'].quantile(q = 0.25))
lower_outlier=related_info['duration'].quantile(q = 0.25)-1.5*(related_info['duration'].quantile(q = 0.75)-related_info['duration'].quantile(q = 0.25))
print("higher_outlier={}\nlower_outlier={}".format(higher_outlier,lower_outlier))

In [ ]:
# 通話時間長さ分割（分割目的はデータ処理しやすい、データ変換処理後に精度が高める、学習時間が短縮する）
#function to creat group of duration, because of duration is importance item and according to grouping seems accuracy improve and decrease training time，Treat outliers separately as a group
# 因为持续时间是重要的项目并且根据分组似乎准确性提高并且减少了训练时间,把异常值单独作为一个组处理
def duration(dataframe):
    dataframe.loc[dataframe['duration'] <= 102, 'duration'] = 1
    dataframe.loc[(dataframe['duration'] > 102) & (dataframe['duration'] <= 180)  , 'duration']    = 2
    dataframe.loc[(dataframe['duration'] > 180) & (dataframe['duration'] <= 319)  , 'duration']   = 3
    dataframe.loc[(dataframe['duration'] > 319) & (dataframe['duration'] <= 644.5), 'duration'] = 4
    dataframe.loc[dataframe['duration']  > 644.5, 'duration'] = 5

duration(related_info);

In [ ]:
fig,ax=plt.subplots()
fig.set_size_inches(10,6)
sns.countplot(x='duration',data=related_info)
ax.set_xlabel("After convert duration distribution chart")
ax.set_title("Duration distribution chart")

### 連絡先、月、day_of_weekフィールドのデータステータスを表示します(Display the data status of contact, month, day_of_week fields)

In [ ]:
fig,(ax1,ax2,ax3)=plt.subplots(nrows=1,ncols=3,figsize=(15,6))
sns.countplot(x='contact',data=related_info,ax=ax1)
sns.countplot(x='month',data=related_info,ax=ax2,order=['mar', 'apr','may', 'jun', 'jul', 'aug','sep', 'oct', 'nov', 'dec'])
sns.countplot(x='day_of_week',data=related_info,ax=ax3)

In [ ]:
related_info_encoder=related_info.copy()
labelencoder=LabelEncoder()
related_info_encoder['contact']=labelencoder.fit_transform(related_info_encoder['contact'])
related_info_encoder['month']=labelencoder.fit_transform(related_info_encoder['month'])
related_info_encoder['day_of_week']=labelencoder.fit_transform(related_info_encoder['day_of_week'])
related_info_encoder['duration']=labelencoder.fit_transform(related_info_encoder['duration'])

In [ ]:
related_info_encoder.head()

### 社会的および経済的背景指標（social and economic context attributes）

In [ ]:
social_info=base_info.iloc[:,15:20]
social_info.head()

In [ ]:
# 経済的背景指標
# emp.var.rate: employment variation rate（就业变化率）
# cons.price.idx: consumer price index（消费者价格指数）
# cons.conf.idx: consumer confidence index（消费者信心指数）
# euribor3m: euribor 3 month rate（欧元3个月利率）
# nr.employed: number of employees（在职员工人数）
# 根据下图，都是数值型数据，不需要转换（According to the figure below, they are all numerical data and do not need to be converted）
fig,(ax1,ax2,ax3,ax4,ax5)=plt.subplots(nrows=1,ncols=5,figsize=(20,6))
sns.set(rc={'figure.figsize':(10,5)})
sns.lineplot(data=social_info['emp.var.rate'],ax=ax1)
ax1.set_xlabel('emp.var.rate')
sns.lineplot(data=social_info['cons.price.idx'],ax=ax2)
ax2.set_xlabel('cons.price.idx')
sns.lineplot(data=social_info['cons.conf.idx'],ax=ax3)
ax3.set_xlabel('cons.conf.idx')
sns.lineplot(data=social_info['euribor3m'],ax=ax4)
ax4.set_xlabel('euribor3m')
sns.lineplot(data=social_info['nr.employed'],ax=ax5)
ax5.set_xlabel('nr.employed')

### 他の指標（other attributes）

In [ ]:
other_info=base_info.iloc[:,11:15]
other_info.head()

```
campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
previous: number of contacts performed before this campaign and for this client (numeric)
poutcome: outcome of the previous marketing campaign (categorical: "failure","nonexistent","success")
```

### campaign項目処理（campaign field processing）

In [ ]:
print(other_info['campaign'].describe())
sns.lineplot(data=other_info['campaign'])

In [ ]:
# 计算outlier范围higher_outlier和lower_outlier（Calculate the outlier range higher_outlier and lower_outlier）
higher_outlier=other_info['campaign'].quantile(q = 0.75)+1.5*(other_info['campaign'].quantile(q = 0.75)-other_info['campaign'].quantile(q = 0.25))
lower_outlier=other_info['campaign'].quantile(q = 0.25)-1.5*(other_info['campaign'].quantile(q = 0.75)-other_info['campaign'].quantile(q = 0.25))
print("higher_outlier={}\nlower_outlier={}".format(higher_outlier,lower_outlier))

In [ ]:
# グループ分けで精度が高めて、学習時間が短縮する。
#function to creat group of campaign. according to grouping seems accuracy improve and decrease training time，Treat outliers separately as a group
# 通过分组准确性提高并且减少了训练时间,把异常值单独作为一个组处理
def campaign(dataframe):
    dataframe.loc[dataframe['campaign'] <= 1, 'campaign'] = 1
    dataframe.loc[(dataframe['campaign'] > 1) & (dataframe['campaign'] <= 2)  , 'campaign']    = 2
    dataframe.loc[(dataframe['campaign'] > 2) & (dataframe['campaign'] <= 3)  , 'campaign']   = 3
    dataframe.loc[(dataframe['campaign'] > 3) & (dataframe['campaign'] <= 6), 'campaign'] = 4
    dataframe.loc[dataframe['campaign']  > 6, 'campaign'] = 5

campaign(other_info);

In [ ]:
# 转换后的campaign状态（Campaign status after conversion）
sns.countplot(x='campaign',data=other_info)

In [ ]:
sns.countplot(x='pdays',data=other_info)

In [ ]:
labelencoder = LabelEncoder()
other_info['poutcome'] = labelencoder.fit_transform(other_info['poutcome']) 

In [ ]:
sns.countplot(x='poutcome',data=other_info)

# 前処理(preprocessing)

In [ ]:
bank= pd.concat([client_info_encoder, related_info_encoder, other_info, social_info], axis = 1)
bank.columns=['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
                     'contact', 'month', 'day_of_week', 'duration','campaign', 'pdays', 'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx', 
                     'cons.conf.idx', 'euribor3m', 'nr.employed']
bank.shape

In [ ]:
columns=['age', 'job', 'marital', 'education', 'default', 'housing', 'loan','contact', 'month', 'day_of_week', 'duration','campaign', 'pdays', 'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed']

In [ ]:
bank.head()

### train,testデータ分割

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(bank, y, test_size = 0.15, random_state = 1234)
k_fold = KFold(n_splits=10, shuffle=True, random_state=0)

In [ ]:
k_fold

In [ ]:
x_train.shape,y_train.shape

In [ ]:
x_train.head()

### データ標準化  
目的：    
データ処理は速くなる。   
データ表現力はアップとなる。   

In [ ]:
sc_X = StandardScaler()
x_train_scale = sc_X.fit_transform(x_train)
x_test_scale = sc_X.transform(x_test)

In [ ]:
x_train_scale_pd=pd.DataFrame(x_train_scale)
x_train_scale_pd.columns=[x_train.columns]
x_train_scale_pd

# 特徴選択(Feature Selection)

In [ ]:
sc_X = StandardScaler()
bank_all_info_scale = sc_X.fit_transform(bank)
bank_all_info_scale=pd.DataFrame(bank_all_info_scale)
bank_all_info_scale.columns=[bank.columns]
bank_all_info_scale

In [ ]:
bank_all_info_scale=pd.concat([bank_all_info_scale,y],axis = 1)
bank_all_info_scale.columns=[base_info.columns]
bank_all_info_scale

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
sns.heatmap(bank_all_info_scale.corr(),
            vmin=-1,
            vmax=1,
            cmap='bwr',
            cbar=True,
            annot=False,
            ax=ax)

### LassoCVで特徴選択

In [ ]:
%%time
from sklearn.linear_model import LassoCV
CV_count=3
lasso = LassoCV(cv=CV_count, random_state=1234).fit(x_train_scale_pd, y_train.values.ravel())
coef = np.abs(lasso.coef_)
coef=coef.reshape(1,-1)
df_importance = pd.DataFrame(coef,columns=bank.columns)
df_importance

### TOP10特徴出力

In [ ]:
# Showing top 10 important features
n_top_features = 10
df_importance_top = df_importance.median().sort_values(ascending=False)[:n_top_features]
name_features = df_importance_top.index
left = name_features
height = df_importance_top.values
plt.bar(left, height,align="center",linewidth=1)
plt.xticks(rotation=270)
plt.title("feature importance")
plt.xlabel("feature variables")
plt.ylabel("degree")
plt.grid(True)
print(name_features)

```
客户信息部分（client）
   1 - age (numeric)
   2 - job : type of job (categorical: "admin.","blue-collar","entrepreneur","housemaid","management","retired","self-employed","services","student","technician","unemployed","unknown")
   3 - marital : marital status (categorical: "divorced","married","single","unknown"; note: "divorced" means divorced or widowed)
   4 - education (categorical: "basic.4y","basic.6y","basic.9y","high.school","illiterate","professional.course","university.degree","unknown")
   5 - default: has credit in default? (categorical: "no","yes","unknown")
   6 - housing: has housing loan? (categorical: "no","yes","unknown")
   7 - loan: has personal loan? (categorical: "no","yes","unknown")
当前广告活动最后联系相关信息（related with the last contact of the current campaign）
   8 - contact: contact communication type (categorical: "cellular","telephone") 
   9 - month: last contact month of year (categorical: "jan", "feb", "mar", ..., "nov", "dec")
  10 - day_of_week: last contact day of the week (categorical: "mon","tue","wed","thu","fri")
  11 - duration: last contact duration, in seconds (numeric). 
其他属性（other attributes）
  12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
  13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
  14 - previous: number of contacts performed before this campaign and for this client (numeric)
  15 - poutcome: outcome of the previous marketing campaign (categorical: "failure","nonexistent","success")
社会和经济状况属性（social and economic context attributes）
  16 - emp.var.rate: employment variation rate（就业变化率） - quarterly indicator (numeric)
  17 - cons.price.idx: consumer price index（消费者价格指数） - monthly indicator (numeric)     
  18 - cons.conf.idx: consumer confidence index（消费者信心指数） - monthly indicator (numeric)     
  19 - euribor3m: euribor 3 month rate（欧元3个月利率） - daily indicator (numeric)
  20 - nr.employed: number of employees（在职员工人数） - quarterly indicator (numeric)
```

### 上記の特徴的な結果から、注文成功率と社会経済的地位の属性の間には大きな相関関係があることがわかります。  

从上面的特征结果可以发现，订购成功率和社会经济状况属性有很大相关性

In [ ]:
x_train_scale = pd.DataFrame(sc_X.fit_transform(x_train[name_features]),
                              columns=x_train[name_features].columns)
x_test_scale = pd.DataFrame(sc_X.transform(x_test[name_features]),
                             columns=x_test[name_features].columns)

In [ ]:
x_train_scale.shape,x_test_scale.shape

# Modeling

### Using LogisticRegression Model

In [ ]:
logmodel = LogisticRegression() 
model=logmodel.fit(x_train_scale,y_train.values.ravel())
logpred = model.predict(x_test_scale)

In [ ]:
from sklearn.metrics import plot_confusion_matrix
print("Prediction accuracy={} %".format(round(accuracy_score(y_test, logpred),2)*100))
LOGCV = (cross_val_score(logmodel, x_train_scale, y_train.values.ravel(), cv=k_fold, n_jobs=1, scoring = 'accuracy').mean())
print("Cross Validation={}".format(LOGCV))

In [ ]:
# confusion matrix chart show
disp = plot_confusion_matrix(model, x_test_scale, y_test,
                                 display_labels=['subscribe','No subscribe'],
                                 cmap=plt.cm.Blues,
                                 normalize=None)
disp.ax_.set_title("Prediction Subscribe Result Show")
print("Prediction Subscribe Result Show")
print(disp.confusion_matrix)
plt.show()

### Using XGBClassifier Model

In [ ]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(x_train_scale, y_train.values.ravel())
xgbprd = xgb.predict(x_test_scale)

In [ ]:
print(confusion_matrix(y_test, xgbprd ))
print(round(accuracy_score(y_test, xgbprd),2)*100)
XGB = (cross_val_score(estimator = xgb, X = x_train_scale, y = y_train.values.ravel(), cv = 10).mean())

In [ ]:
# confusion matrix chart show
disp = plot_confusion_matrix(xgb,x_test_scale, y_test,
                                 display_labels=['subscribe','No subscribe'],
                                 cmap=plt.cm.Blues,
                                 normalize=None)
# disp.ax_.set_title("Prediction Subscribe Result Show")
print("Prediction Subscribe Result Show")
print(disp.confusion_matrix)
plt.show()

### Using SVC Model

### HyperParameter tuning of SVC

In [ ]:
%%time

import time
from sklearn import svm
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from sklearn.model_selection import cross_val_score, StratifiedKFold


def objective(params):
    params = {
        'C': abs(float(params['C'])),
        "kernel": str(params['kernel']),
    }
    clf = svm.SVC(gamma='scale', **params)

    #     采用前5000条数据,数据量进行超参数优化测试（Using the first 5000 data,data volume for hyperparameter optimization test）
    score = -np.mean(
        cross_val_score(clf,
                        x_train_scale[:10000],
                        y_train.values.ravel()[:10000],
                        cv=3,
                        n_jobs=-1,
                        scoring="neg_mean_squared_error"))

    print("loss score {:.3f} params {}".format(score, params))
    return {'loss': score, 'status': STATUS_OK}


start_time = time.time()
space = {
    'C': hp.normal('C', 0, 50),
    "kernel": hp.choice('kernel', ['rbf']),
}

best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=5)
kernel = ['rbf']
best['kernel'] = kernel[best['kernel']]
best['C'] = abs(best['C'])
hyperparametertuning_time = round(time.time() - start_time, 2)
print(
    "SVC: Hyperopt estimated optimum {}. Hyper parameter tuning spend time {}".
    format(best, hyperparametertuning_time))

In [ ]:
%%time
start_time = time.time()
svc = svm.SVC(gamma='scale', **best)
svc.fit(x_train_scale,y_train.values.ravel())
training_time = round(time.time() - start_time, 2)
print("SVC: Training spend time {}".format(training_time))

### Prediction of SVC

In [ ]:
svcprd = svc.predict(x_test_scale)
print(confusion_matrix(y_test, svcprd))
print("Prediction accuracy={} %".format(round(accuracy_score(y_test, svcprd),2)*100))
SVC = (cross_val_score(logmodel, x_train_scale, y_train.values.ravel(), cv=k_fold, n_jobs=1, scoring = 'accuracy').mean())
print("Cross Validation={}".format(SVC))

In [ ]:
# confusion matrix chart show
disp = plot_confusion_matrix(svc, x_test_scale, y_test,
                                 display_labels=['subscribe','No subscribe'],
                                 cmap=plt.cm.Blues,
                                 normalize=None)
disp.ax_.set_title("Prediction Subscribe Result Show")
print("Prediction Subscribe Result Show")
print(disp.confusion_matrix)
plt.show()

# 結論（Conclusion）

In [ ]:
models = pd.DataFrame({
                'Models': ['Logistic Classifier', 'XGBoost','SVClassifier' ],
                'Score':  [LOGCV,XGB,SVC]})

models.sort_values(by='Score', ascending=False)